In [1]:
import cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
def mean_image(folder_path, max_files=-1):
    image_paths = glob(folder_path + "*.tif")[:max_files]
    first_time = True
    for path in image_paths:
        print(path)
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if first_time:
            first_time = False
            mean_img = img.astype(np.float64)
        else:
            mean_img += img
    return (mean_img/len(image_paths)).astype(np.uint8)

def detect_colored_noise(mean_img, channel_thresholds=(15, 15, 15)):
    blurred = cv2.GaussianBlur(mean_img, (5, 5), 0)
    diff = cv2.absdiff(mean_img, blurred)
    b, g, r = cv2.split(diff)
    _, hot_b = cv2.threshold(b, channel_thresholds[0], 255, cv2.THRESH_BINARY)
    _, hot_g = cv2.threshold(g, channel_thresholds[1], 255, cv2.THRESH_BINARY)
    _, hot_r = cv2.threshold(r, channel_thresholds[2], 255, cv2.THRESH_BINARY)
    hot_rgb = cv2.bitwise_or(hot_r, cv2.bitwise_or(hot_g, hot_b))
    print("hot pixels, r: {}".format(hot_r[hot_r>0].size))
    print("hot pixels, g: {}".format(hot_g[hot_g>0].size))
    print("hot pixels, b: {}".format(hot_b[hot_b>0].size))
    print("hot pixels, rgb: {}".format(hot_rgb[hot_rgb>0].size))
    return hot_rgb, (hot_b, hot_g, hot_r)

In [3]:
def correct_defective_pixels(image, defect_mask, kernel_size=3, method='mean'):
    if len(image.shape) == 3:
        corrected = image.copy()
        for c in range(3):
            channel = image[:, :, c]
            defect_channel = defect_mask > 0
            corrected[:, :, c] = _correct_channel(channel, defect_channel, kernel_size, method)
    else:
        corrected = _correct_channel(image, defect_mask > 0, kernel_size, method)
    return corrected
    
def _correct_channel(channel, defect_mask, kernel_size, method):
    corrected = channel.copy()
    defect_coords = np.argwhere(defect_mask)
    for y, x in defect_coords:
        neighborhood = channel[
            max(0, y - kernel_size//2):min(channel.shape[0], y + kernel_size//2 + 1),
            max(0, x - kernel_size//2):min(channel.shape[1], x + kernel_size//2 + 1)
        ]        
        valid_pixels = neighborhood[neighborhood != 0]  # Opzionale: escludi altri pixel difettosi se necessario
        if len(valid_pixels) > 0:
            if method == 'mean':
                corrected[y, x] = np.mean(valid_pixels)
            elif method == 'median':
                corrected[y, x] = np.median(valid_pixels)
    return corrected

In [4]:
input_path = "E:/Focus stacking/2025-04-21 - Hemiptera/J/src/"
mean_img = mean_image(input_path, 100)
hot_rgb, (hot_b, hot_g, hot_r) = detect_colored_noise(mean_img)

E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9376.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9377.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9378.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9379.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9380.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9381.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9382.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9383.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9384.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9385.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9386.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9387.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9388.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9389.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9390.tif
E:/Focus stacking/2025-04-21 - Hemiptera/J/src\_MG_9391.tif
E:/Focus stacking/2025-04-21 - Hemiptera

In [6]:
out_path = "E:/Focus stacking/2025-04-21 - Hemiptera/J/noise/"
cv2.imwrite(out_path + "hot_b.png", hot_b)
cv2.imwrite(out_path + "hot_g.png", hot_g)
cv2.imwrite(out_path + "hot_r.png", hot_r)
cv2.imwrite(out_path + "hot_rgb.png", hot_rgb)
cv2.imwrite(out_path + "mean_img.tif", mean_img)

True

In [7]:
image = cv2.imread(input_path + "/_MG_9376.tif")
defect_mask = cv2.imread(out_path + "hot_rgb.png", cv2.IMREAD_GRAYSCALE)  # Maschera binaria
corrected_image = correct_defective_pixels(image, defect_mask, kernel_size=5, method='median')
cv2.imwrite(out_path + "orig.tif", image)
cv2.imwrite(out_path + "corr.tif", corrected_image)

True